# Import modules

In [1]:
!pip install cython_bbox

     |████████████████████████████████| 51kB 2.3MB/s 
  Created wheel for cython-bbox: filename=cython_bbox-0.1.3-cp36-cp36m-linux_x86_64.whl size=57159 sha256=e15a27b4fb10bcb738d7999fd887dd8a71a69bb46341bad9d0a864f1c4ba8a9d
  Stored in directory: /root/.cache/pip/wheels/2b/31/b5/9246d5988e79ef89dc28b894835d2f305e23c1e5f4f80278ee
Successfully built cython-bbox


In [2]:
%tensorflow_version 1.x
%matplotlib inline

from glob import glob
from imageio import imread
from tqdm import tqdm
from cython_bbox import bbox_overlaps
import tensorflow as tf
import numpy as np
import numpy.random as npr
import matplotlib.pyplot as plt
import os.path as osp
import json
import cv2
import os

# deprecated
import tensorflow.contrib.slim as slim

TensorFlow 1.x selected.
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



# Copy data

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [4]:
!rm -rf ./data
!cp -r '/content/drive/My Drive/2020_04_CSF_Detection/data_0818' ./data

# Load data

In [5]:
# Get a list of the class names.
def read_classes(ds_dir):
    cls_names = []

    with open(osp.join(ds_dir, 'classes.json'), 'r') as f:
        cls_dict = json.load(f)
    for i in sorted(cls_dict.keys(), key=int):
        cls_names.append(cls_dict[str(i)])

    return cls_names


# Load image files.
def read_imgs(ds_dir):
    imgs = []

    img_fpaths = sorted(glob(osp.join(ds_dir, 'images/*.jpg')))
    num_samples = len(img_fpaths)

    for img_fp in tqdm(img_fpaths):
        img = imread(img_fp)
        if len(img.shape) < 3:
            img = cv2.cvtColor(img, cv2.COLOR_GRAY2RGB)

        imgs.append(np.asarray(img / 255, dtype=np.float32))

    files = [os.path.basename(f) for f in img_fpaths]
    return files, imgs


In [6]:
DS_DIR = './data/test'
#DS_DIR = './data/external'

cls_names = read_classes(DS_DIR)
num_obj_classes = len(cls_names)

print(cls_names)
print(num_obj_classes)

['Erythrocyte', 'Lymphocyte', 'Mononuclear cell', 'Neutrophil']
4


# Hyper-parameters for Faster R-CNN inference

In [8]:
ANCHOR_BASE_SIZE = 16
ANCHOR_SCALES = [3, 6]
ANCHOR_RATIOS = [0.67, 1, 1.5]

K = len(ANCHOR_SCALES) * len(ANCHOR_RATIOS)

NMS_TOP_K = 300  # for inference
NMS_IOU_THRES = 0.7

ROI_POOL_SIZE = 7
RPN_CHANNELS = 512
TOTAL_STRIDE = 16

RPN_POS_RATIO = 0.5
RPN_BATCH_SIZE = 128
RPN_NEG_THRES = 0.3
RPN_POS_THRES = 0.7

FRCNN_POS_RATIO = 0.25
FRCNN_BATCH_SIZE = 128
FRCNN_POS_THRES = 0.7
FRCNN_NEG_THRES = 0.5

LOG_DIR = 'exp'
EXP_NAME = 'pilot_0604'

COLOR_MAP = {
    0: (1, 0, 0),     # Erythrocyte
    1: (0, 1, 0),     # Lymphocyte
    2: (0, 0, 1),     # Mononuclear
    3: (0.7, 0.7, 0)  # Neutrophil
}

# Define functions

## Anchor box functions

In [9]:
def _to_whxy_format(anchor):
    """
    Return width, height, x center, and y center for an anchor (window).
    """
    w = anchor[2] - anchor[0] + 1
    h = anchor[3] - anchor[1] + 1
    x_ctr = anchor[0] + 0.5 * (w - 1)
    y_ctr = anchor[1] + 0.5 * (h - 1)
    return w, h, x_ctr, y_ctr


def _to_anchor_format(ws, hs, x_ctr, y_ctr):
    """
    Given a vector of widths (ws) and heights (hs) around a center (x_ctr, y_ctr), output a set of anchors (windows).
    """
    ws = ws[:, np.newaxis]
    hs = hs[:, np.newaxis]
    anchors = np.hstack(
        (x_ctr - 0.5 * (ws - 1), y_ctr - 0.5 * (hs - 1), x_ctr + 0.5 * (ws - 1), y_ctr + 0.5 * (hs - 1))
    )
    return anchors


def _scale_enum(anchor, scales):
    """
    Enumerate a set of anchors for each scale wrt an anchor.
    """
    scales = np.array(scales)
    w, h, x_ctr, y_ctr = _to_whxy_format(anchor)
    ws = w * scales
    hs = h * scales
    anchors = _to_anchor_format(ws, hs, x_ctr, y_ctr)
    return anchors


def _ratio_enum(anchor, ratios):
    """
    Enumerate a set of anchors for each aspect ratio wrt an anchor.
    """
    ratios = np.array(ratios)
    w, h, x_ctr, y_ctr = _to_whxy_format(anchor)
    size = w * h
    size_ratios = size / ratios
    ws = np.round(np.sqrt(size_ratios))
    hs = np.round(ws * ratios)
    anchors = _to_anchor_format(ws, hs, x_ctr, y_ctr)
    return anchors


def generate_anchor_boxes(input_image, total_stride, anchor_base_size, anchor_scales, anchor_ratios):
    image_shape = tf.shape(input_image)
    
    # Calculate the number of grid cells.
    num_grids_h = tf.to_int32(tf.ceil(image_shape[1] / np.float32(total_stride)))
    num_grids_w = tf.to_int32(tf.ceil(image_shape[2] / np.float32(total_stride)))
    num_grids = num_grids_h * num_grids_w
    
    t_checkup['num_grids_h'] = num_grids_h
    t_checkup['num_grids_w'] = num_grids_w
    t_checkup['num_grids'] = num_grids
    
    # Prepare a shift expansion.
    shift_x, shift_y = tf.meshgrid(
        tf.range(num_grids_w) * total_stride, tf.range(num_grids_h) * total_stride
    )
    shift_x, shift_y = tf.reshape(shift_x, (-1,)), tf.reshape(shift_y, (-1,))  # [num_grids], [num_grids]
    shifts = tf.transpose(tf.stack([shift_x, shift_y, shift_x, shift_y]))  # [num_grids, 4]
    shifts = tf.transpose(tf.reshape(shifts, shape=[1, num_grids, 4]), perm=(1, 0, 2))  # [num_grids, 1, 4]
    
    # Create a base anchor.
    base_anchor = np.array([0, 0, anchor_base_size - 1, anchor_base_size - 1])  # [4]
    
    # Expand anchors by ratios.
    anchors = _ratio_enum(base_anchor, anchor_ratios)  # [num_ratios, 4]
        
    # Expand anchors by scales.
    anchors = np.vstack(  # [num_ratios * num_scales, 4]
        [_scale_enum(anchors[i, :], anchor_scales) for i in range(anchors.shape[0])]
    )
    
    # Expand anchors by shifts.
    num_anchors_per_grid = anchors.shape[0]
    anchors = tf.constant(anchors.reshape((1, num_anchors_per_grid, 4)), dtype=tf.int32)  # [1, num_anchors_per_grid, 4]
    
    shift_x, shift_y = tf.meshgrid(
        tf.range(num_grids_w) * total_stride, tf.range(num_grids_h) * total_stride
    )
    shift_x, shift_y = tf.reshape(shift_x, (-1,)), tf.reshape(shift_y, (-1,))  # [num_grids], [num_grids]
    shifts = tf.transpose(tf.stack([shift_x, shift_y, shift_x, shift_y]))  # [num_grids, 4]
    shifts = tf.transpose(tf.reshape(shifts, shape=[1, num_grids, 4]), perm=(1, 0, 2))  # [num_grids, 1, 4]
    
    anchors = tf.add(anchors, shifts)  # [num_grids, num_anchors_per_grid, 4]
    anchors = tf.reshape(anchors, (-1, 4))  # [num_anchors, 4], where num_grids * num_anchors_per_grid = num_anchors
    
    return tf.cast(anchors, dtype=tf.float32)


## Bounding box functions

In [10]:
# Fot test only. 
# Use cython's bbox_overlaps instead!
def my_bbox_overlaps(boxes1, boxes2):
    """Compute the IOUs between the two sets of boxes.
    
    Args:
        boxes1 (numpy.ndarray): [num_boxes1, 4]-D array
        boxes2 (numpy.ndarray): [num_boxes2, 4]-D array
    
    Each of a box comprises 4 coordinate values in [xmin, ymin, xmax, ymax] order.
        
    Returns:
        overlaps (numpy.ndarray): [num_boxes1, num_boxes2]-D array, which is the distance matrix of the two sets of boxes
    """
    # Compute the areas of `boxes1` and `boxes2`.
    area1 = (boxes1[:, 2] - boxes1[:, 0] + 1) * (boxes1[:, 3] - boxes1[:, 1] + 1)  # [num_boxes1]
    area2 = (boxes2[:, 2] - boxes2[:, 0] + 1) * (boxes2[:, 3] - boxes2[:, 1] + 1)  # [num_boxes2]
    
    # Compute the areas of the intersections.
    intersection_h = np.maximum(
        (np.minimum(np.expand_dims(boxes1[:, 3], axis=1), boxes2[:, 3]) -
         np.maximum(np.expand_dims(boxes1[:, 1], axis=1), boxes2[:, 1]) + 1),
        0
    )  # [num_boxes1, num_boxes2]-D
    intersection_w = np.maximum(
        (np.minimum(np.expand_dims(boxes1[:, 2], axis=1), boxes2[:, 2]) -
         np.maximum(np.expand_dims(boxes1[:, 0], axis=1), boxes2[:, 0]) + 1),
        0
    )  # [num_boxes1, num_boxes2]-D
    intersection = intersection_h * intersection_w  # [num_boxes1, num_boxes2]-D

    # Compute the areas of the unions.
    union = np.maximum(
        np.expand_dims(area1, 1) + area2 - intersection,
        np.finfo(float).eps
    )
    
    # Compute IOU values.
    iou = intersection / union

    return iou


def compute_bbox_deltas(src_bboxes, dst_bboxes):
    src_widths = src_bboxes[:, 2] - src_bboxes[:, 0] + 1.0
    src_heights = src_bboxes[:, 3] - src_bboxes[:, 1] + 1.0
    src_ctr_x = src_bboxes[:, 0] + 0.5 * src_widths
    src_ctr_y = src_bboxes[:, 1] + 0.5 * src_heights

    dst_widths = dst_bboxes[:, 2] - dst_bboxes[:, 0] + 1.0
    dst_heights = dst_bboxes[:, 3] - dst_bboxes[:, 1] + 1.0
    dst_ctr_x = dst_bboxes[:, 0] + 0.5 * dst_widths
    dst_ctr_y = dst_bboxes[:, 1] + 0.5 * dst_heights

    targets_dx = (dst_ctr_x - src_ctr_x) / src_widths
    targets_dy = (dst_ctr_y - src_ctr_y) / src_heights
    targets_dw = np.log(dst_widths / src_widths)
    targets_dh = np.log(dst_heights / src_heights)

    targets = np.vstack((targets_dx, targets_dy, targets_dw, targets_dh)).transpose()
    return targets


def _unmap(data, count, indices, fill=0):
    """Unmap a subset of item (data) back to the original set of items (of size count)
    """
    if len(data.shape) == 1:
        ret = np.empty((count,), dtype=np.float32)
        ret.fill(fill)
        ret[indices] = data
    else:
        ret = np.empty((count,) + data.shape[1:], dtype=np.float32)
        ret.fill(fill)
        ret[indices, :] = data
    return ret


def transform_bboxes(boxes, deltas):
    """
    Args:
        boxes: [num_boxes, 4], (xmin, ymin, xmax, ymax) formed
        deltas: [num_boxes, 4], (tx, ty, tw, th)
    """
    boxes = tf.cast(boxes, deltas.dtype)
    
    # Compute size and center coordinates of the boxex.
    widths = tf.subtract(boxes[:, 2], boxes[:, 0]) + 1.0
    heights = tf.subtract(boxes[:, 3], boxes[:, 1]) + 1.0
    ctr_x = tf.add(boxes[:, 0], widths * 0.5)
    ctr_y = tf.add(boxes[:, 1], heights * 0.5)

    tx, ty, tw, th = deltas[:, 0], deltas[:, 1], deltas[:, 2], deltas[:, 3]

    transformed_ctr_x = tf.add(tf.multiply(tx, widths), ctr_x)
    transformed_ctr_y = tf.add(tf.multiply(ty, heights), ctr_y)
    transformed_w = tf.multiply(tf.exp(tw), widths)
    transformed_h = tf.multiply(tf.exp(th), heights)

    transformed_xmin = tf.subtract(transformed_ctr_x, transformed_w * 0.5)
    transformed_ymin = tf.subtract(transformed_ctr_y, transformed_h * 0.5)
    transformed_xmax = tf.add(transformed_ctr_x, transformed_w * 0.5)
    transformed_ymax = tf.add(transformed_ctr_y, transformed_h * 0.5)

    return tf.stack([transformed_xmin, transformed_ymin, transformed_xmax, transformed_ymax], axis=1)


def clip_bboxes(boxes, img_shape):
    """
    Args:
        boxes: [n_boxes, 4]
        img_shape: [2]
    """
    img_shape = tf.to_float(img_shape)
    b0 = tf.maximum(tf.minimum(boxes[:, 0], img_shape[2] - 1), 0)
    b1 = tf.maximum(tf.minimum(boxes[:, 1], img_shape[1] - 1), 0)
    b2 = tf.maximum(tf.minimum(boxes[:, 2], img_shape[2] - 1), 0)
    b3 = tf.maximum(tf.minimum(boxes[:, 3], img_shape[1] - 1), 0)
    
    return tf.stack([b0, b1, b2, b3], axis=1)


def transform_bboxes_np(boxes, deltas):
    """
    Args:
        boxes: [num_boxes, 4], (xmin, ymin, xmax, ymax) formed
        deltas: [num_boxes, 4], (tx, ty, tw, th)
    """
    # Compute size and center coordinates of the boxex.
    widths = boxes[:, 2] -  boxes[:, 0] + 1.0
    heights = boxes[:, 3] - boxes[:, 1] + 1.0
    ctr_x = boxes[:, 0] + widths * 0.5
    ctr_y = boxes[:, 1] + heights * 0.5

    tx, ty, tw, th = deltas[:, 0], deltas[:, 1], deltas[:, 2], deltas[:, 3]

    transformed_ctr_x = (tx * widths) + ctr_x
    transformed_ctr_y = (ty * heights) + ctr_y
    transformed_w = np.exp(tw) * widths
    transformed_h = np.exp(th) * heights

    transformed_xmin = transformed_ctr_x - transformed_w * 0.5
    transformed_ymin = transformed_ctr_y - transformed_h * 0.5
    transformed_xmax = transformed_ctr_x + transformed_w * 0.5
    transformed_ymax = transformed_ctr_y + transformed_h * 0.5

    return np.stack([transformed_xmin, transformed_ymin, transformed_xmax, transformed_ymax], axis=1)


def clip_bboxes_np(boxes, img_shape):
    b0 = np.maximum(np.minimum(boxes[:, 0], img_shape[1] - 1), 0)
    b1 = np.maximum(np.minimum(boxes[:, 1], img_shape[0] - 1), 0)
    b2 = np.maximum(np.minimum(boxes[:, 2], img_shape[1] - 1), 0)
    b3 = np.maximum(np.minimum(boxes[:, 3], img_shape[0] - 1), 0)
    
    return np.stack([b0, b1, b2, b3], axis=1)


def nms(boxes, rpn_cls_probs, top_k, iou_thres):
    obj_scores = tf.reshape(tf.transpose(tf.reshape(rpn_cls_probs, (-1, K, 2)), (0, 2, 1)), tf.shape(rpn_cls_probs))[..., K:]
    nms_indices = tf.image.non_max_suppression(
        boxes, tf.reshape(obj_scores, (-1,)), max_output_size=top_k, iou_threshold=iou_thres
    )
    boxes_alive = tf.gather(boxes, nms_indices)
    scores_alive = tf.gather(tf.reshape(obj_scores, (-1,)), nms_indices)
    
    return boxes_alive, scores_alive


def cpu_nms(boxes, iou_thres=0.5):
    x1 = boxes[..., 0]
    y1 = boxes[..., 1]
    x2 = boxes[..., 2]
    y2 = boxes[..., 3]
    areas = (x2 - x1 + 1) * (y2 - y1 + 1)
    scores = boxes[..., 4]

    keep = []
    order = scores.argsort()[::-1]

    while order.size > 0:
        i = order[0]
        keep.append(i)
        xx1 = np.maximum(x1[i], x1[order[1:]])
        yy1 = np.maximum(y1[i], y1[order[1:]])
        xx2 = np.minimum(x2[i], x2[order[1:]])
        yy2 = np.minimum(y2[i], y2[order[1:]])

        w = np.maximum(0.0, xx2 - xx1 + 1)
        h = np.maximum(0.0, yy2 - yy1 + 1)
        inter = w * h
        ovr = inter / (areas[i] + areas[order[1:]] - inter)
        inds = np.where(ovr <= iou_thres)[0]
        order = order[inds + 1]

    return keep


## Backbone CNN (VGG-16) defining function

In [11]:
def conv_layers(input_image, is_training):
    """
    Args:
        image (np.ndarray): [1, image_height, image_width, image_channel]
        is_training (bool)
    
    Returns:
        conv_feats (Tensor): [1, height/stride, width/stride, depth]
    """
    net = slim.repeat(input_image, 2, slim.conv2d, 64, [3, 3], trainable=is_training, scope='conv1')
    net = slim.max_pool2d(net, [2, 2], padding='SAME', scope='pool1')  # total stride: 2
    net = slim.repeat(net, 2, slim.conv2d, 128, [3, 3], trainable=is_training, scope='conv2')
    net = slim.max_pool2d(net, [2, 2], padding='SAME', scope='pool2')  # total stride: 4
    net = slim.repeat(net, 3, slim.conv2d, 256, [3, 3], trainable=is_training, scope='conv3')
    net = slim.max_pool2d(net, [2, 2], padding='SAME', scope='pool3')  # total stride: 8
    net = slim.repeat(net, 3, slim.conv2d, 512, [3, 3], trainable=is_training, scope='conv4')
    net = slim.max_pool2d(net, [2, 2], padding='SAME', scope='pool4')  # total stride: 16
    net = slim.repeat(net, 3, slim.conv2d, 512, [3, 3], trainable=is_training, scope='conv5')
    conv_feats = net
    return conv_feats


## RPN layer defining functions

In [12]:
def rpn_intermediate_layer(conv_feats, rpn_channels, is_training):
    rpn_intermediate = slim.conv2d(conv_feats, rpn_channels, [3, 3], trainable=is_training)
    return rpn_intermediate


def rpn_classification_layer(rpn_intermediate, is_training):
    cls_scores = slim.conv2d(  # [H/16, W/16, 2 * num_anchors_per_grid]
        rpn_intermediate,
        K * 2, [1, 1], trainable=is_training,
        weights_initializer=tf.random_normal_initializer(mean=0.0, stddev=0.01),
        weights_regularizer=tf.contrib.layers.l2_regularizer(0.0001),
        padding='VALID', activation_fn=None
    )
    cls_probs = tf.reshape(tf.nn.softmax(tf.reshape(cls_scores, (-1, 2))), tf.shape(cls_scores))
    cls_preds = tf.argmax(tf.reshape(cls_probs, (-1, 2)), axis=1)
    return cls_scores, cls_probs, cls_preds


def rpn_regression_layer(rpn_intermediate, is_training):
    bbox_params = slim.conv2d(
        rpn_intermediate,
        K * 4, [1, 1], trainable=is_training,
        weights_initializer=tf.random_normal_initializer(mean=0.0, stddev=0.01),
        weights_regularizer=tf.contrib.layers.l2_regularizer(0.0001),
        padding='VALID', activation_fn=None
    )
    return bbox_params


## Detection layer (Fast R-CNN) defining functions

In [13]:
def roi_pooling_layer(conv_feats, proposal_boxes, pool_size):
    # Get normalized ROI coordinates.
    ceiled_img_h = tf.to_float(tf.shape(t_conv_feats)[1]) * np.float32(TOTAL_STRIDE)
    ceiled_img_w = tf.to_float(tf.shape(t_conv_feats)[2]) * np.float32(TOTAL_STRIDE)                
    xmin = proposal_boxes[:, 0:1] / ceiled_img_w
    ymin = proposal_boxes[:, 1:2] / ceiled_img_h
    xmax = proposal_boxes[:, 2:3] / ceiled_img_w
    ymax = proposal_boxes[:, 3:4] / ceiled_img_h
    normalized_boxes = tf.concat([ymin, xmin, ymax, xmax], axis=1)  # [num_proposal_boxes, 4]

    batch_indices = tf.zeros((tf.shape(normalized_boxes)[0],), dtype=tf.int32)

    pre_pool_size = pool_size * 2
    cropped_feats = tf.image.crop_and_resize(
        conv_feats, normalized_boxes, batch_indices, [pre_pool_size, pre_pool_size], name='roi_pooled_feats'
    )
    pooled_feats = slim.max_pool2d(cropped_feats, [2, 2], padding='SAME')
    
    return pooled_feats


def fc_layers(roi_pooled_feats, is_training):
    """
    Args:
        poold_rois: [num_rois, roi_height, roi_width, image_channel]
        is_training (bool)
        
    Returns:
        fc_feats (Tensor): [num_rois, depth]
    """
    pooled_rois_flat = slim.flatten(roi_pooled_feats, scope='pooled_rois_flat')
    fc6 = slim.fully_connected(pooled_rois_flat, 4096, scope='fc6', reuse=tf.AUTO_REUSE)
    dropout6 = slim.dropout(fc6, keep_prob=0.5, is_training=is_training, scope='dropout6')
    fc7 = slim.fully_connected(fc6, 4096, scope='fc7', reuse=tf.AUTO_REUSE)
    dropout7 = slim.dropout(fc7, keep_prob=0.5, is_training=is_training, scope='dropout7')
    fc_feats = dropout7

    return fc_feats


def faster_rcnn_classification_layer(fc_feats, num_obj_classes, is_training):
    num_classes = num_obj_classes + 1
    
    cls_scores = slim.fully_connected(
        fc_feats, num_classes,
        weights_initializer=tf.random_normal_initializer(mean=0.0, stddev=0.01),
        trainable=is_training, activation_fn=None, scope='cls_scores', reuse=tf.AUTO_REUSE
    )
    cls_probs = tf.nn.softmax(cls_scores)
    cls_preds = tf.argmax(cls_probs, axis=1)
    
    return cls_scores, cls_probs, cls_preds


def faster_rcnn_regression_layer(fc_feats, num_obj_classes, is_training):
    num_classes = num_obj_classes + 1
    
    bbox_params = slim.fully_connected(
        fc_feats, num_classes * 4,
        weights_initializer=tf.random_normal_initializer(mean=0.0, stddev=0.001),
        trainable=is_training, activation_fn=None, scope='bbox_params', reuse=tf.AUTO_REUSE
    )
    
    return bbox_params


## End-to-end detection function

In [14]:
def detect(img):
    img_batch = np.expand_dims(img, axis=0)

    feed_dict = {t_input['image']: img_batch}
    proposal_boxes, cls_probs, bbox_params = sess.run(
        [t_proposal_boxes, t_frcnn_cls_probs, t_frcnn_bbox_params], feed_dict=feed_dict
    )

    num_preds = bbox_params.shape[0]
    cls_preds = np.argmax(cls_probs, axis=1)
    bbox_params = np.reshape(bbox_params, (num_preds, -1, 4))
    bbox_params_selected = np.empty((num_preds, 4), dtype=np.float32)
    for i in range(num_preds):
        cls_idx = cls_preds[i]
        bbox_params_selected[i, :] = bbox_params[i, cls_idx]

    pred_boxes = transform_bboxes_np(proposal_boxes, bbox_params_selected)
    pred_boxes = clip_bboxes_np(pred_boxes, img.shape)

    final_boxes = []
    for i in range(1, num_obj_classes+1):
        pb = np.append(pred_boxes[cls_preds==i], cls_probs[cls_preds==i][..., i:i+1], axis=1)
        nms_boxes_idx = cpu_nms(pb, 0.5)
        nms_boxes = pb[nms_boxes_idx][..., 0:4].astype(int)

        final_boxes = final_boxes + [nb.tolist() + [i-1] for nb in nms_boxes]
    
    return final_boxes


# Build a tensorflow model

## Placeholders

In [15]:
t_input = {
    'image': tf.placeholder(tf.float32, shape=(1, None, None, 3)),  # [height, width, channel]
    'gt_box_set': tf.placeholder(tf.float32, shape=[1, None, 5])  # [[xmin, ymin, xmax, ymax, cls_idx], ...]
}

t_checkup = {}
loss_checkup = {}

## CNN feature extraction

In [16]:
# final feature map of CNN
t_conv_feats = conv_layers(t_input['image'], False)


Instructions for updating:
Please use `layer.__call__` method instead.


## RPN classification & regression

In [17]:
t_rpn_intermediate = rpn_intermediate_layer(t_conv_feats, RPN_CHANNELS, False)

t_rpn_cls_scores, t_rpn_cls_probs, t_rpn_cls_preds = rpn_classification_layer(t_rpn_intermediate, False)
t_rpn_bbox_params = rpn_regression_layer(t_rpn_intermediate, False)

## Proposal box tensor (anchor + bbox_param)

In [18]:
# anchor boxes
t_anchor_boxes = generate_anchor_boxes(t_input['image'], TOTAL_STRIDE, ANCHOR_BASE_SIZE, ANCHOR_SCALES, ANCHOR_RATIOS)

# transform anchor boxes wrt bbox params
t_transformed_anchor_boxes = transform_bboxes(t_anchor_boxes, tf.reshape(t_rpn_bbox_params, (-1, 4)))
t_clipped_anchor_boxes = clip_bboxes(t_transformed_anchor_boxes, tf.shape(t_input['image']))

# region proposal boxes
t_proposal_boxes, t_proposal_scores = nms(t_clipped_anchor_boxes, t_rpn_cls_probs, NMS_TOP_K, NMS_IOU_THRES)

Instructions for updating:
Use `tf.cast` instead.
Instructions for updating:
Use `tf.cast` instead.


## Detection (Fast R-CNN) classification & regression

In [19]:
t_roi_pooled_feats = roi_pooling_layer(t_conv_feats, t_proposal_boxes, ROI_POOL_SIZE)
t_fc_feats = fc_layers(t_roi_pooled_feats, False)

t_frcnn_cls_scores, t_frcnn_cls_probs, t_frcnn_cls_preds = faster_rcnn_classification_layer(t_fc_feats, num_obj_classes, False)
t_frcnn_bbox_params = faster_rcnn_regression_layer(t_fc_feats, num_obj_classes, False)

Instructions for updating:
box_ind is deprecated, use box_indices instead
Instructions for updating:
Use keras.layers.flatten instead.


# Create a tensorflow session

In [20]:
sess = tf.InteractiveSession()
sess.run(tf.global_variables_initializer())

# Restore the trained model

In [21]:
exp_dir_GD = osp.join('/content/drive/My Drive/2020_04_CSF_Detection', LOG_DIR, EXP_NAME)
exp_dir = osp.join('.', LOG_DIR)

!rm -rf '$exp_dir'
!mkdir '$exp_dir'
!cp -r '$exp_dir_GD' '$exp_dir'

exp_dir = osp.join('.', LOG_DIR, EXP_NAME)

In [22]:
saver = tf.train.Saver()
saver.restore(sess, osp.join(exp_dir, 'trained_model'))

INFO:tensorflow:Restoring parameters from ./exp/pilot_0604/trained_model


# Test on all test images

In [23]:
files, imgs = read_imgs(DS_DIR)

num_imgs = len(imgs)
print('\nnum of images:', num_imgs)
print(files)

100%|██████████| 60/60 [00:00<00:00, 67.21it/s]


num of images: 60
['47927_region_0.jpg', '47927_region_1.jpg', '47927_region_2.jpg', '47927_region_3.jpg', '47927_region_4.jpg', '47927_region_5.jpg', '47927_region_6.jpg', 'B01562_region_0.jpg', 'B01562_region_1.jpg', 'B01562_region_10.jpg', 'B01562_region_11.jpg', 'B01562_region_12.jpg', 'B01562_region_2.jpg', 'B01562_region_3.jpg', 'B01562_region_4.jpg', 'B01562_region_5.jpg', 'B01562_region_6.jpg', 'B01562_region_7.jpg', 'B01562_region_8.jpg', 'B01562_region_9.jpg', 'B01778_region_0.jpg', 'B01778_region_1.jpg', 'B01778_region_10.jpg', 'B01778_region_11.jpg', 'B01778_region_12.jpg', 'B01778_region_2.jpg', 'B01778_region_3.jpg', 'B01778_region_4.jpg', 'B01778_region_5.jpg', 'B01778_region_6.jpg', 'B01778_region_7.jpg', 'B01778_region_8.jpg', 'B01778_region_9.jpg', 'B02530_region_0.jpg', 'B02530_region_1.jpg', 'B02530_region_2.jpg', 'B02530_region_3.jpg', 'B02530_region_4.jpg', 'B02530_region_5.jpg', 'B02530_region_6.jpg', 'B02530_region_7.jpg', 'B02530_region_8.jpg', 'B02530_region_

In [24]:
detection_dict = {}

for file, img in zip(tqdm(files), imgs):
    det_boxes = detect(img)
    detection_dict[file] = (img, det_boxes)


100%|██████████| 60/60 [00:33<00:00,  1.82it/s]


In [26]:
MAX_LIMIT = 10

plt.style.use(['default'])

for i, file in enumerate(detection_dict):
    if i >= MAX_LIMIT:
        break

    img, det_boxes = detection_dict[file]

    img_show = img.copy()
    for box in det_boxes:
        img_show = cv2.rectangle(img_show, (box[0], box[1]), (box[2], box[3]), color=COLOR_MAP[box[4]], thickness=2)

    plt.figure(figsize=(10, 10))
    plt.imshow((img_show * 255).astype(np.uint8))
    plt.show()


Output hidden; open in https://colab.research.google.com to view.

# Interactive Viewing

## GUI functions

In [27]:
import ipywidgets as widgets
from ipywidgets import interact, interactive, interact_manual


In [28]:
detection_dict = {}

def on_button_clicked(b):
    for file, img in zip(tqdm(files), imgs):
        det_boxes = detect(img)
        detection_dict[file] = (img, det_boxes)
    print(f'\n{len(files)} files detection completed.')

run_button = widgets.Button(description='Run Detection')
run_button.on_click(on_button_clicked)


In [29]:
def select_and_show(file, erythrocyte, lymphocyte, mononuclear, neutrophil):
    img, bboxes = detection_dict[file]

    show_switches = [erythrocyte, lymphocyte, mononuclear, neutrophil]

    img_show = img.copy()
    for box in bboxes:
        c = box[4]
        if show_switches[c]:
            cv2.rectangle(img_show, (box[0], box[1]), (box[2], box[3]), color=COLOR_MAP[c], thickness=2)

    plt.figure(figsize=(14, 14))
    plt.imshow(img_show)
    plt.show()


selector = interactive(
    select_and_show,
    file=files,
    erythrocyte=False,
    lymphocyte=False,
    mononuclear=False,
    neutrophil=False)


## Display GUI

In [30]:
display(run_button)

Button(description='Run Detection', style=ButtonStyle())

100%|██████████| 60/60 [00:04<00:00, 14.06it/s]


60 files detection completed.


In [31]:
display(selector)

interactive(children=(Dropdown(description='file', options=('47927_region_0.jpg', '47927_region_1.jpg', '47927…

# Finish.

Ian Animal Medical Center's CSF Detection Model (Test version 1.0 ==> v0.0.1)